In [11]:
# Dependencies 
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import glob as glob

In [12]:
# Set path of html file
races_html_path = 'Race_Results_HTML/stpetersburg.htm'
# Read html file and store as html_content
with open(races_html_path 'r') as races_file:
    html_content = races_file.read()

In [13]:
soup = BeautifulSoup(races_html_content, 'html.parser')

In [14]:
# Get the full-season table from HTML
season_html = soup.find(id = 'race-alltime')
season_data = []
# Loop through table html and parse to list
for row in season_html.find_all('tr'):
    row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
    season_data.append(row_data)
season_df = pd.DataFrame(season_data[1:], columns=season_data[0])
season_df["Date"] = pd.to_datetime(season_df["Date"])
season_df = season_df.drop([1, 2])
season_df = season_df.drop("Avg.Speed", axis=1).drop("Winner", axis=1).drop("Car/Engine", axis=1).drop("Start Position", axis=1)
season_df.reset_index(drop=True, inplace=True)
season_df.index += 1
season_df

,Date,Track,Track Type
1,2024-03-10,Streets of St. Petersburg,Street Course
2,2024-03-24,The Thermal Club,Road Course
3,2024-04-21,Streets of Long Beach,Street Course
4,2024-04-28,Barber Motorsports Park,Road Course
5,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course
6,2024-05-26,Indianapolis Motor Speedway,Oval
7,2024-06-02,Streets of Detroit,Street Course
8,2024-06-09,Road America,Road Course
9,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course
10,2024-07-07,Mid-Ohio Sports Car Course,Road Course


In [15]:
# Data Cleaning specific to 2024 season
# Correct the incorrect date of Iowa race 1
season_df.at[11, 'Date'] = '2024-07-13'
# Create row for shorthand race name
season_df.at[1, 'Race'] = 'St. Pete'
season_df.at[2, 'Race'] = 'Thermal'
season_df.at[3, 'Race'] = 'Long Beach'
season_df.at[4, 'Race'] = 'Barber'
season_df.at[5, 'Race'] = 'Indy Road Course'
season_df.at[6, 'Race'] = 'Indy 500'
season_df.at[7, 'Race'] = 'Detroit'
season_df.at[8, 'Race'] = 'Road America'
season_df.at[9, 'Race'] = 'Laguna Seca'
season_df.at[10, 'Race'] = 'Mid-Ohio'
season_df.at[11, 'Race'] = 'Iowa 1'
season_df.at[12, 'Race'] = 'Iowa 2'
season_df.at[13, 'Race'] = 'Toronto'
season_df.at[14, 'Race'] = 'Gateway'
season_df.at[15, 'Race'] = 'Portland'
season_df.at[16, 'Race'] = 'Milwaukee 1'
season_df.at[17, 'Race'] = 'Milwaukee 2'
season_df.at[18, 'Race'] = 'Nashville'
season_df

,Date,Track,Track Type,Race
1,2024-03-10,Streets of St. Petersburg,Street Course,St. Pete
2,2024-03-24,The Thermal Club,Road Course,Thermal
3,2024-04-21,Streets of Long Beach,Street Course,Long Beach
4,2024-04-28,Barber Motorsports Park,Road Course,Barber
5,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course,Indy Road Course
6,2024-05-26,Indianapolis Motor Speedway,Oval,Indy 500
7,2024-06-02,Streets of Detroit,Street Course,Detroit
8,2024-06-09,Road America,Road Course,Road America
9,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course,Laguna Seca
10,2024-07-07,Mid-Ohio Sports Car Course,Road Course,Mid-Ohio


In [19]:
all_files_path = 'Race_Results_HTML/*.htm'
race_results_files = []
race_results_files = glob.glob(all_files_path)

season_races_df = pd.DataFrame()

for race_result_file in race_results_files:
# Read html file and store as html_content
    with open(race_result_file, 'r') as file:
        html_content = file.read()
    # Parse race result table html
    soup = BeautifulSoup(html_content, 'html.parser')
    race_table_html = soup.find(id = 'race-season')
# Initialize a list to contain the race results 
    race_result_data = []
# Loop through table html and parse to list
    for row in race_table_html.find_all('tr'):
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
        race_result_data.append(row_data)

    race_result_df = pd.DataFrame(race_result_data[1:], columns=race_result_data[0])
    race_name = soup.find(id = 'season-event-name').text
    race_date = soup.find(id = 'season-session-date').text
    race_date = datetime.strptime(race_date, '%A, %B %d, %Y').date()
    race_winner = race_result_df.loc[race_result_df['Rank'] == '1', 'Driver'].iloc[0]
    race_pole = race_result_df.loc[race_result_df['Starts'] == '1', 'Driver'].iloc[0]
    race_avg_speed = race_result_df.loc[race_result_df['Rank'] == '1', 'Avg.Speed'].iloc[0]
    race_total_time = race_result_df.loc[race_result_df['Rank'] == '1', 'Total time'].iloc[0]
    race_laps = race_result_df.loc[race_result_df['Rank'] == '1', 'Laps'].iloc[0]
    race_info = {'Date': [race_date],
            'Race Name (Official)': [race_name],
            'Laps': [race_laps],
            'Winner': [race_winner],
            'Pole': [race_pole],
            'Avg. Speed': [race_avg_speed],
            'Total Time': [race_total_time],
            'Results': [race_result_df]}
    race_df = pd.DataFrame(race_info)
    race_df["Date"] = pd.to_datetime(race_df["Date"])
    race_df.reset_index(drop=True, inplace=True)
    race_df.index += 1
    season_races_df = pd.concat([season_races_df, race_df], axis=0)

season_races_df

,Date,Race Name (Official),Laps,Winner,Pole,Avg. Speed,Total Time,Results
1,2024-06-02,Chevrolet Detroit Grand Prix,100,Scott Dixon,Colton Herta,78.251,02:06:07.9684,Rank Driver Car No. Starts La...
1,2024-04-21,Acura Grand Prix of Long Beach,85,Scott Dixon,Felix Rosenqvist,98.350,01:42:03.1416,Rank Driver Car No. Starts La...
1,2024-06-09,XPEL Grand Prix at Road America,55,Will Power,Linus Lundqvist,126.154,01:45:00.0267,Rank Driver Car No. Starts La...
1,2024-07-13,Hy-Vee One Step 250,250,Will Power,Scott McLaughlin,154.768,01:26:38.7472,Rank Driver Car No. Starts La...
1,2024-07-14,Hy-Vee Homefront 250,250,Scott McLaughlin,Colton Herta,128.098,01:44:41.1172,Rank Driver Car No. Starts La...
1,2024-03-10,Firestone Grand Prix of St. Petersburg,100,Pato O'Ward,Josef Newgarden,96.752,01:51:37.5075,Rank Driver Car No. Starts La...
1,2024-08-25,BITNILE.COM Grand Prix of Portland,110,Will Power,Santino Ferrucci,112.161,01:55:34.1948,Rank Driver Car No. Starts La...
1,2024-07-07,Honda Indy 200 at Mid-Ohio,80,Pato O'Ward,Alex Palou,116.071,01:33:22.6191,Rank Driver Car No. Starts La...
1,2024-05-26,108th Running of the Indianapolis 500,200,Josef Newgarden,Scott McLaughlin,167.763,02:58:49.4079,Rank Driver Car No. Starts La...
1,2024-05-11,Sonsio Grand Prix,85,Alex Palou,Alex Palou,117.956,01:45:27.2320,Rank Driver Car No. Starts La...


In [17]:
season_full_df = pd.merge(season_df, season_races_df, on="Date", how = "inner")
season_full_df.reset_index(drop=True, inplace=True)
season_full_df.index += 1
season_full_df

,Date,Track,Track Type,Race,Race Name (Official),Laps,Winner,Pole,Avg. Speed,Total Time,Results
1,2024-03-10,Streets of St. Petersburg,Street Course,St. Pete,Firestone Grand Prix of St. Petersburg,100,Pato O'Ward,Josef Newgarden,96.752,01:51:37.5075,Rank Driver Car No. Starts La...
2,2024-04-21,Streets of Long Beach,Street Course,Long Beach,Acura Grand Prix of Long Beach,85,Scott Dixon,Felix Rosenqvist,98.350,01:42:03.1416,Rank Driver Car No. Starts La...
3,2024-04-28,Barber Motorsports Park,Road Course,Barber,Children’s of Alabama Indy Grand Prix,90,Scott McLaughlin,Scott McLaughlin,106.369,01:56:45.7773,Rank Driver Car No. Starts La...
4,2024-05-11,Indianapolis Motor Speedway Road Course,Road Course,Indy Road Course,Sonsio Grand Prix,85,Alex Palou,Alex Palou,117.956,01:45:27.2320,Rank Driver Car No. Starts La...
5,2024-05-26,Indianapolis Motor Speedway,Oval,Indy 500,108th Running of the Indianapolis 500,200,Josef Newgarden,Scott McLaughlin,167.763,02:58:49.4079,Rank Driver Car No. Starts La...
6,2024-06-02,Streets of Detroit,Street Course,Detroit,Chevrolet Detroit Grand Prix,100,Scott Dixon,Colton Herta,78.251,02:06:07.9684,Rank Driver Car No. Starts La...
7,2024-06-09,Road America,Road Course,Road America,XPEL Grand Prix at Road America,55,Will Power,Linus Lundqvist,126.154,01:45:00.0267,Rank Driver Car No. Starts La...
8,2024-06-23,WeatherTech Raceway Laguna Seca,Road Course,Laguna Seca,Firestone Grand Prix of Monterey,95,Alex Palou,Alex Palou,102.740,02:04:09.8545,Rank Driver Car No. Starts La...
9,2024-07-07,Mid-Ohio Sports Car Course,Road Course,Mid-Ohio,Honda Indy 200 at Mid-Ohio,80,Pato O'Ward,Alex Palou,116.071,01:33:22.6191,Rank Driver Car No. Starts La...
10,2024-07-13,Iowa Speedway,Oval,Iowa 1,Hy-Vee One Step 250,250,Will Power,Scott McLaughlin,154.768,01:26:38.7472,Rank Driver Car No. Starts La...
